In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-350gyjte
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-350gyjte
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=d1d0d9dfb3a241b337f5d2b37329a9ca52765921a545d8c80f0430f2e8fb5dcc
  Stored in directory: /tmp/pip-ephem-wheel-cache-v7q9p3je/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name test.cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>

#define N 4092
__global__ void division(int* d_data) {

    int tid = blockDim.x * threadIdx.y + threadIdx.x;
    int num_threads_in_a_block = blockDim.x * blockDim.y;
    int block_offset = blockIdx.x * num_threads_in_a_block;
    int num_threads_in_a_row = num_threads_in_a_block * gridDim.x;
    int row_offset = num_threads_in_a_row * blockIdx.y;
    int gid = tid + block_offset + row_offset;

    //printf("%d,",gid);
    if (gid < N){
        d_data[gid] = gid / 256;
    }
    //printf("blockIdx.x : %d, blockIdx.y : %d, blockDim.x : %d, blockDim.y : %d, GridDim.x : %d, GridDim.y : %d threadIdx.x : %d,threadIdx.y: %d, gid : %d - data : %d \n", blockIdx.x, blockIdx.y,blockDim.x,blockDim.y,gridDim.x,gridDim.y, threadIdx.x,threadIdx.y, gid, d_data[gid]);
}

int main() {
    int *h_data, *d_data;
    int dataSize = sizeof(int) * N;

    h_data= (int*)malloc(sizeof(int)*N);

    cudaMalloc((void**)&d_data, dataSize);
    cudaMemcpy(d_data,h_data,dataSize,cudaMemcpyHostToDevice);

    int nx = 256;
    int ny = 16;

    dim3 block(128, 4, 1);
    dim3 grid(nx/block.x, ny/block.y);

    division<<<grid,block>>>(d_data);

    cudaMemcpy(h_data,d_data,sizeof(int)*N, cudaMemcpyDeviceToHost);
 

    std::cout << std::endl;
    std::cout << "Results: ";
    for (int i=0; i<N; ++i) {
        std::cout << h_data[i] << " ";
    }
    std::cout << std::endl;
    
    free(h_data);

    // Free cuda pointers
    cudaFree(d_data);
    cudaDeviceReset();
    return 0;
}

'File written in /content/src/test.cu'

In [ ]:
!nvcc src/test.cu -o test

In [ ]:
!nvprof ./test

==213== NVPROF is profiling process 213, command: ./test

Results: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [ ]:
%%cuda --name test2.cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>

#define N 4092
__global__ void division(int* d_data) {

    int tid = blockDim.x * threadIdx.y + threadIdx.x;
    int num_threads_in_a_block = blockDim.x * blockDim.y;
    int block_offset = blockIdx.x * num_threads_in_a_block;
    int num_threads_in_a_row = num_threads_in_a_block * gridDim.x;
    int row_offset = num_threads_in_a_row * blockIdx.y;
    int gid = tid + block_offset + row_offset;
    printf("%d,",gid);
    if (gid < N){
        d_data[gid] = blockIdx.x;
    }
    printf("blockIdx.x : %d, blockIdx.y : %d, blockDim.x : %d, blockDim.y : %d, GridDim.x : %d, GridDim.y : %d threadIdx.x : %d,threadIdx.y: %d, gid : %d - data : %d \n", blockIdx.x, blockIdx.y,blockDim.x,blockDim.y,gridDim.x,gridDim.y, threadIdx.x,threadIdx.y, gid, d_data[gid]);
}

int main() {
    int *h_data, *d_data;
    int dataSize = sizeof(int) * N;

    h_data= (int*)malloc(sizeof(int)*N);

    cudaMalloc((void**)&d_data, dataSize);
    cudaMemcpy(d_data,h_data,dataSize,cudaMemcpyHostToDevice);

    int nx = 256;
    int ny = 16;

    dim3 block(256,1);
    dim3 grid(16,1);

    division<<<grid,block>>>(d_data);

    cudaMemcpy(h_data,d_data,sizeof(int)*N, cudaMemcpyDeviceToHost);

    std::cout << std::endl;
    std::cout << "Results: ";
    for (int i=0; i<N; ++i) {
        std::cout << h_data[i] << " ";
    }
    std::cout << std::endl;
    
    free(h_data);

    // Free cuda pointers
    cudaFree(d_data);
    cudaDeviceReset();
    return 0;
}

'File written in /content/src/test2.cu'

In [ ]:
!nvcc src/test2.cu -o test2

src/test2.cu(30): warning: variable "nx" was declared but never referenced

src/test2.cu(31): warning: variable "ny" was declared but never referenced



In [ ]:
!nvprof ./test2

==327== NVPROF is profiling process 327, command: ./test2
blockIdx.x : 10, blockIdx.y : 0, blockDim.x : 256, blockDim.y : 1, GridDim.x : 16, GridDim.y : 1 threadIdx.x : 128,threadIdx.y: 0, gid : 2688 - data : 10 
blockIdx.x : 10, blockIdx.y : 0, blockDim.x : 256, blockDim.y : 1, GridDim.x : 16, GridDim.y : 1 threadIdx.x : 129,threadIdx.y: 0, gid : 2689 - data : 10 
blockIdx.x : 10, blockIdx.y : 0, blockDim.x : 256, blockDim.y : 1, GridDim.x : 16, GridDim.y : 1 threadIdx.x : 130,threadIdx.y: 0, gid : 2690 - data : 10 
blockIdx.x : 10, blockIdx.y : 0, blockDim.x : 256, blockDim.y : 1, GridDim.x : 16, GridDim.y : 1 threadIdx.x : 131,threadIdx.y: 0, gid : 2691 - data : 10 
blockIdx.x : 10, blockIdx.y : 0, blockDim.x : 256, blockDim.y : 1, GridDim.x : 16, GridDim.y : 1 threadIdx.x : 132,threadIdx.y: 0, gid : 2692 - data : 10 
blockIdx.x : 10, blockIdx.y : 0, blockDim.x : 256, blockDim.y : 1, GridDim.x : 16, GridDim.y : 1 threadIdx.x : 133,threadIdx.y: 0, gid : 2693 - data : 10 
blockIdx.x :